# JODA 2022 Oppimispäiväkirja

## Week 1

-Created Git Repository: https://github.com/htaim/JODA2022

-This notebook is available on: https://github.com/htaim/JODA2022/blob/main/Oppimispaivakirja.ipynb